In [ ]:
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd

T = 1
N = 252
dt = 1/N
end = datetime.now()
start = end - timedelta(days=365)
n_paths = 10_000
window=60

print(start, end)

In [ ]:
data = yf.Ticker("TSLA").history(start=start, end=end)

In [ ]:
def simulate_corelated_brownians(T, N, dt, n_paths, rho=-0.5):
  sqrt_dt = np.sqrt(dt)

  Z1 = np.random.randn(n_paths, N)
  Z2 = np.random.randn(n_paths, N)

  dW1 = sqrt_dt * Z1
  dW2 = sqrt_dt * (rho * Z1 + np.sqrt(1.0 - rho**2) *  Z2)

  W1 = np.concatenate([np.zeros((n_paths, 1)), dW1.cumsum(axis=1)], axis=1)
  W2 = np.concatenate([np.zeros((n_paths, 1)), dW2.cumsum(axis=1)], axis=1)

  return W1, W2

def get_kappa_theta(dt, x_window, y_window):

  tmp = pd.concat([x_window, y_window], axis=1).dropna()
  x, y = tmp.iloc[:, 0], tmp.iloc[:, 1]

  model = sm.OLS(y, sm.add_constant(x)).fit()
  a, b = model.params

  k = -np.log(b) / dt
  t = a / (1-b)

  print(f"a={a:.6f}, b={b:.6f}, kappa={k:.4f}, theta={t:.6f}")

  return (k, t)

def simulate_paths(v0, S0, dt, mu, kappa, theta, eps, n_paths, N, Z1, Z2):
  S = np.zeros((n_paths, N+1))
  v = np.zeros((n_paths, N+1))

  S[:, 0] = S0
  v[:, 0] = v0

  for t in range(N):
    v_prev = v[:, t]
    v_sqrt = np.sqrt(v_prev)
    v[:, t+1] = (
        v_prev + kappa * (theta - v_prev) * dt
        + eps * v_sqrt * np.sqrt(dt) * Z2[:, t]
    )
    v[:, t+1] = np.maximum(v[:, t+1], 0)

    S[:, t+1] = S[:, t] * np.exp(
        (mu - 0.5 * v_prev) * dt + np.sqrt(v_prev * dt) * Z1[:, t]
    )

  return S, v

def plot_samples(S, N, dt, n_plots=10):
  t = np.linspace(0, N*dt, N+1)
  for i in range(n_plots):
    plt.plot(t, S[i, :], lw=0.8, alpha=0.7)
  plt.title("Sample Simulated Heston Price Paths")
  plt.xlabel("Time (years)")
  plt.ylabel("Asset Price")
  plt.show()


def plot_mean(S, N, dt):
  t = np.linspace(0, N*dt, N+1)
  mean_S = S.mean(axis=0)
  var_S = S.std(axis=0)

  def show_mean():
    plt.plot(t, mean_S, label="Mean Path", color="red")
  
  def show_confidence():
    plt.fill_between(t, mean_S-var_S, mean_S+var_S, color="green", alpha=0.2, label="±1 Std Dev")

  show_mean()
  show_confidence()

  plt.title("Average Heston Price Path ± 1σ")
  plt.xlabel("Time (years)")
  plt.ylabel("Asset Price")
  plt.legend()
  plt.grid(True)
  plt.show()




In [ ]:
returns = data["Close"].pct_change().dropna()
v_series = returns.rolling(window=window).var().dropna()

mu = float(returns.mean() * N)

x_window = v_series.iloc[:-1]
y_window = v_series.shift(-1).iloc[:-1]

kappa, theta = get_kappa_theta(dt, x_window, y_window)

dv = v_series.diff().dropna()
eps = dv.std() * np.sqrt(252)

S0 = float(data["Close"].iloc[-1])
v0 = float((v_series.iloc[-1]))

Z1, Z2 = simulate_corelated_brownians(T, N, dt, n_paths=n_paths)

emp_rho = np.corrcoef(Z1[:, -1], Z2[:, -1])[0,1]
print("Empirical correlation at T:", emp_rho)

S, v = simulate_paths(v0, S0, dt, mu, kappa, theta, eps, n_paths, N, Z1, Z2)

plot_samples(S, N, dt)
plot_mean(S, N, dt)
